In [48]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import signal
import seaborn as sns

In [49]:
import scikit_posthocs as sp
import numpy as np
from scipy import stats
import json

In [50]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
Daily_level_Data=pd.read_csv("Data_Daily_level.csv")
Daily_level_Data.head(2)

,Date_Daily_Level,Average Signups,Active %,CPA,Average observed profit,Estimated LTV,New Users,Users,Bounces,Pageviews,Page Load Time (ms),ROAS (up to date)_smoothened,ROAS (estimated)_smoothened
0,2021-08-01,1101,0.022707,31.362341,136.178029,143.416200,NaN,NaN,NaN,NaN,NaN,4.232135,4.437028
1,2021-08-02,802,0.017456,39.260330,143.763778,149.335326,NaN,NaN,NaN,NaN,NaN,4.055284,4.244659


In [52]:
Monthly_level_Data=pd.read_csv("Data_Monthly_level.csv")
for column in Monthly_level_Data.columns[1:]:
    Monthly_level_Data.loc[:,column]=Monthly_level_Data[column].apply(lambda x:x.strip('][').replace("'","").split(','))
    Monthly_level_Data.loc[:,column]=Monthly_level_Data[column].apply(lambda x:[temp.strip() for temp in x])
    try:
        Monthly_level_Data.loc[:,column]=Monthly_level_Data[column].apply(lambda x:[float(temp) for temp in x])
    except:
        pass
Monthly_level_Data.head(2)

,Date_Monthly_Level,Operating System,Region,Users_1,Bounces_1,Sessions_1,Pageviews_1,Page Load Time (ms)_1,Signups_1,Source / Medium,Users_3,Bounces_3,Sessions_3,Pageviews_3,Page Load Time (ms)_3,Signups_3
0,2021-09-01,"[iOS, Android, iOS, iOS, iOS, iOS, iOS, iOS, i...","[Texas, Texas, California, Florida, Georgia, N...","[28670.0, 15790.0, 14785.0, 14520.0, 13700.0, ...","[18461.0, 9601.0, 8490.0, 7697.0, 8014.0, 6691...","[43508.0, 27110.0, 23830.0, 20550.0, 19307.0, ...","[124863.0, 105767.0, 73553.0, 59694.0, 55436.0...","[5045524.0, 6669434.0, 2814674.0, 1961975.0, 1...","[1798.0, 1137.0, 397.0, 661.0, 714.0, 714.0, 7...","[fb / paidsocial, (direct) / (none), m.faceboo...","[170831.0, 90031.0, 74029.0, 34436.0, 33431.0,...","[98944.0, 46391.0, 39355.0, 19413.0, 19307.0, ...","[189927.0, 179744.0, 83128.0, 72971.0, 60117.0...","[365730.0, 804695.0, 173899.0, 336213.0, 24782...","[26251890.0, 10469393.0, 8633498.0, 7425305.0,...","[11399.0, 1878.0, 6268.0, 1587.0, 2248.0, 1719..."
1,2021-10-01,"[iOS, Android, iOS, iOS, iOS, iOS, iOS, Androi...","[Texas, Texas, North Carolina, Georgia, Califo...","[24385.0, 17800.0, 11637.0, 11346.0, 11082.0, ...","[13542.0, 10209.0, 6850.0, 6877.0, 6162.0, 629...","[36975.0, 28326.0, 18091.0, 16319.0, 17165.0, ...","[120367.0, 99552.0, 54775.0, 49009.0, 52844.0,...","[5250262.0, 1053655.0, 2683780.0, 1331258.0, 1...","[1375.0, 1349.0, 397.0, 450.0, 582.0, 661.0, 7...","[fb / paidsocial, (direct) / (none), m.faceboo...","[140574.0, 92279.0, 56388.0, 39382.0, 39329.0,...","[78975.0, 47792.0, 29940.0, 14811.0, 22005.0, ...","[156945.0, 178395.0, 63979.0, 44090.0, 84397.0...","[311193.0, 804298.0, 125974.0, 113226.0, 40754...","[14928369.0, 20728177.0, 4494073.0, 7272989.0,...","[9759.0, 1878.0, 3571.0, 3385.0, 2592.0, 1851...."


# 2. Exploratory Data Analysis (EDA)

## Trend and Pattern in Advertising Performance & Number of Sign-Ups

**It can be investigated from 3 prespectives:**
1. Source/Medium
2. Operating System
3. Region

### Source/Medium

1. Source/Medium will be standardized based on major sources(facebook,google,etc..) and data is grouped at this level to analyze trends and patterns.
**Note** : 
<br>a. At an advanced level, segmentation is performed at a detailed level on every major Source 
<br>b. Data starts at september and ends at June. So analysis can only be performed on : Monthly Basis or Quarterly Basis and with extra time frame, we can perform reasonable analysis at a yearly level. 
<br>c. Multiple Sources/Medium have exactly the same numbers of sign ups (26) so this issue is investigated if the the number is actually identical or it is a problem related to data collection
<br>d. There are Sources with no sign-up so they will be ignored

In [53]:
Medium_source=Monthly_level_Data[["Date_Monthly_Level","Source / Medium","Signups_3"]]
Medium_source=Medium_source.explode(["Source / Medium","Signups_3"])
Medium_source.head()

,Date_Monthly_Level,Source / Medium,Signups_3
0,2021-09-01,fb / paidsocial,11399.0
0,2021-09-01,(direct) / (none),1878.0
0,2021-09-01,m.facebook.com / referral,6268.0
0,2021-09-01,google / organic,1587.0
0,2021-09-01,google / cpc,2248.0


In [54]:
Medium_source=Medium_source[~Medium_source["Signups_3"].isna()]
Medium_source["Source / Medium"]=Medium_source["Source / Medium"].apply(lambda x:x.split("/")[0])
Medium_source["Source / Medium"]=Medium_source["Source / Medium"].apply(lambda x:x.replace(".com",""))
Medium_source["Source / Medium"]=Medium_source["Source / Medium"].apply(lambda x:x.replace("fb","facebook"))
Medium_source["Source / Medium"]=Medium_source["Source / Medium"].apply(lambda x:x.replace("lm.",""))
Medium_source["Source / Medium"]=Medium_source["Source / Medium"].apply(lambda x:x.replace("l.",""))
Medium_source["Source / Medium"]=Medium_source["Source / Medium"].apply(lambda x:x.replace("m.",""))
Medium_source["Source / Medium"]=Medium_source["Source / Medium"].apply(lambda x:x.replace("mail.",""))
Medium_source["Source / Medium"]=Medium_source["Source / Medium"].apply(lambda x:x.strip())

In [55]:
sign_ups=Medium_source[["Date_Monthly_Level","Signups_3"]].groupby(["Date_Monthly_Level"]).sum().reset_index()
sign_ups=sign_ups.sort_values(by=["Date_Monthly_Level"])
fig = px.line(sign_ups, x="Date_Monthly_Level", y="Signups_3", title='Monthly Sign Up Count')
fig.show()

* There is an overall increasing trend in number of sign-ups starting from November 2022. 
* Now, let's Check the Top 10 Sources and thier corrpesonding percentages

In [56]:
Counter=Medium_source.groupby(["Source / Medium"]).sum().reset_index()[["Source / Medium","Signups_3"]].sort_values(by=["Signups_3"],ascending=False)

In [57]:
fig = px.pie(Counter.head(10), values='Signups_3', names='Source / Medium')
fig.show()

* The top 3 Sources are :
<br> Facebook
<br> Google
<br> (direct)
<br> so we will analyze their corresponding time series data to take at a look at their performance at a more detailed level

In [58]:
filteration_condition=Medium_source["Source / Medium"]=="facebook"
filteration_condition_1=Medium_source["Source / Medium"]=="google"
filteration_condition_2=Medium_source["Source / Medium"]=="(direct)"

In [59]:
Facebook=Medium_source[filteration_condition].groupby(["Date_Monthly_Level","Source / Medium"]).sum().reset_index()
Google=Medium_source[filteration_condition_1].groupby(["Date_Monthly_Level","Source / Medium"]).sum().reset_index()
Direct=Medium_source[filteration_condition_2].groupby(["Date_Monthly_Level","Source / Medium"]).sum().reset_index()

In [60]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Facebook["Date_Monthly_Level"], y=Facebook["Signups_3"],mode='lines',name='Facebook'))
fig.add_trace(go.Scatter(x=Google["Date_Monthly_Level"], y=Google["Signups_3"],mode='lines',name='Google'))
fig.add_trace(go.Scatter(x=Direct["Date_Monthly_Level"], y=Direct["Signups_3"],mode='lines',name='Direct'))

fig.show()

* The number of sign-ups coming from Direct Source is stable with higher fluctuation from Google and the highest fluctuation in Facebook. 
* Facebook Sign-ups started to increase from November but these customers have low potential of buying because the sign-up increase wasn't translated to an increase in corresponding ROAS, instead it was declining

## Website Traffic Data

* We will analyze Website Traffic Data based on :
<br> Operationg System 
<br> Region
* To Uderstand User Engagmenet , we will work on 3 Main Metrics
<br> New Users
<br> Users
<br> Sessions

### a.Based on Operating System

In [61]:
dataframe_temp_1=Monthly_level_Data[["Operating System","Users_1","Signups_1","Sessions_1","Pageviews_1"]]
dataframe_temp_1=dataframe_temp_1.explode(["Operating System","Users_1","Signups_1","Sessions_1","Pageviews_1"])

In [62]:
dataframe_temp_OS=dataframe_temp_1.groupby(["Operating System"]).sum().reset_index()
dataframe_temp_OS=dataframe_temp_OS.sort_values(by=["Users_1","Signups_1","Sessions_1","Pageviews_1"],ascending=False)
dataframe_temp_OS

,Operating System,Users_1,Signups_1,Sessions_1,Pageviews_1
3,iOS,5033790.0,245508.0,6524354.0,19150665.0
0,Android,3294069.0,237524.0,4837730.0,17154097.0
1,Linux,4111.0,338.0,6317.0,18758.0
2,Macintosh,391.0,0.0,525.0,1690.0


In [63]:
Operating_System=list(dataframe_temp_OS["Operating System"].unique())

fig = go.Figure(data=[
    go.Bar(name="Users", x=Operating_System, y=dataframe_temp_OS["Users_1"].values),
    go.Bar(name="SignUps", x=Operating_System, y=dataframe_temp_OS["Signups_1"].values),
    go.Bar(name="Sessions", x=Operating_System, y=dataframe_temp_OS["Sessions_1"].values),
    go.Bar(name="PageViews", x=Operating_System, y=dataframe_temp_OS["Pageviews_1"].values)
 
])
fig.update_layout(barmode='group')
fig.show()

* From the above Graph, we can identify that IOS and Android Operating system presents most of the traffic sources and user engagements . There can be two reasons :
<br>1. Android and iOS present the highest traffic data sources
<br>2. There isn't enough information collected from the other two data source and extra investigation needs to be made on data source collection process. I am more inclined with the second approach becaus most of the months don't contain data about Linux and Macintosh
<br>We will analyze the top two operating system trends over time 

In [64]:
dataframe_temp_2=Monthly_level_Data.explode(["Operating System","Users_1","Signups_1","Sessions_1","Pageviews_1","Bounces_1"])
dataframe_temp_2=dataframe_temp_2[["Date_Monthly_Level","Operating System","Users_1","Signups_1","Sessions_1","Pageviews_1","Bounces_1"]]
dataframe_temp_2=dataframe_temp_2.groupby(["Date_Monthly_Level","Operating System"]).sum().reset_index()
dataframe_temp_2=dataframe_temp_2[dataframe_temp_2["Operating System"].isin(["Android","iOS"])]
dataframe_temp_2.head()

,Date_Monthly_Level,Operating System,Users_1,Signups_1,Sessions_1,Pageviews_1,Bounces_1
0,2021-09-01,Android,159609.0,12352.0,260857.0,962959.0,92723.0
1,2021-09-01,iOS,281667.0,15628.0,397803.0,1188696.0,158025.0
2,2021-10-01,Android,196686.0,14094.0,302483.0,1073725.0,112955.0
4,2021-10-01,iOS,239854.0,12483.0,351570.0,1150557.0,129248.0
5,2021-11-01,Android,122135.0,8038.0,202861.0,874965.0,71620.0


In [65]:
Monthly_level_Android=dataframe_temp_2[dataframe_temp_2["Operating System"]=="Android"]["Date_Monthly_Level"].values
OS_Android_Users=dataframe_temp_2[dataframe_temp_2["Operating System"]=="Android"]["Users_1"].values
OS_Android_SignUps=dataframe_temp_2[dataframe_temp_2["Operating System"]=="Android"]["Signups_1"].values
OS_Android_Sessions=dataframe_temp_2[dataframe_temp_2["Operating System"]=="Android"]["Sessions_1"].values
OS_Android_PageViews=dataframe_temp_2[dataframe_temp_2["Operating System"]=="Android"]["Pageviews_1"].values
OS_Android_Bounces=dataframe_temp_2[dataframe_temp_2["Operating System"]=="Android"]["Bounces_1"].values

In [66]:
Monthly_level_iOS=dataframe_temp_2[dataframe_temp_2["Operating System"]=="iOS"]["Date_Monthly_Level"].values
OS_iOS_Users=dataframe_temp_2[dataframe_temp_2["Operating System"]=="iOS"]["Users_1"].values
OS_iOS_SignUps=dataframe_temp_2[dataframe_temp_2["Operating System"]=="iOS"]["Signups_1"].values
OS_iOS_Sessions=dataframe_temp_2[dataframe_temp_2["Operating System"]=="iOS"]["Sessions_1"].values
OS_iOS_PageViews=dataframe_temp_2[dataframe_temp_2["Operating System"]=="iOS"]["Pageviews_1"].values
OS_iOS_Bounces=dataframe_temp_2[dataframe_temp_2["Operating System"]=="iOS"]["Bounces_1"].values

## Users

In [67]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Monthly_level_Android, y=OS_Android_Users,mode='lines+markers',name="Users_Android"))
fig.add_trace(go.Scatter(x=Monthly_level_iOS, y=OS_iOS_Users,mode='lines+markers',name="Usrs_IoS"))
fig.show()

## Sign Ups

In [68]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Monthly_level_Android, y=OS_Android_SignUps,mode='lines+markers',name='SignUps_Android'))
fig.add_trace(go.Scatter(x=Monthly_level_iOS, y=OS_iOS_SignUps,mode='lines+markers',name='SignUps_iOS'))
fig.show()

## Sessions

In [69]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Monthly_level_Android, y=OS_Android_Sessions,mode='lines+markers',name='Sessions_Android'))
fig.add_trace(go.Scatter(x=Monthly_level_iOS, y=OS_iOS_Sessions,mode='lines+markers',name='Sessions_iOS'))
fig.show()

## Page Views

In [70]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Monthly_level_Android, y=OS_Android_PageViews,mode='lines+markers',name='PageViews_Android'))
fig.add_trace(go.Scatter(x=Monthly_level_iOS, y=OS_iOS_PageViews,mode='lines+markers',name='PageViews_iOS'))
fig.show()

## Bounces

In [71]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Monthly_level_Android, y=OS_Android_Bounces,mode='lines+markers',name='Bounces_Android'))
fig.add_trace(go.Scatter(x=Monthly_level_iOS, y=OS_iOS_Bounces,mode='lines+markers',name='Bounces_iOS'))
fig.show()

### a.Based on Region

* There are three different types stated in the column Region : 
<br> City 
<br> Country
<br> Sub-Country
We will merge the three levels with an external data source to get the corresponding country and plot a geo-spatial plot. 
* **Note** :
1. There are regions that can exist in different countries so they are dropped as extra information is needed to understand the corresponding count. (In Total, there are 358 Regions, out of them we can determine 288 Regions.
2. Data Can be aggregated at a City or Subcountry level but for simplicity we will do it for now at a Country Level
3. There are multiple cities where Values are exactly identical so more investigation needs to be made for this data source

1. First Data Source

In [72]:
database_world=pd.read_csv("world-cities.csv")
database_name=database_world[["name","country"]].drop_duplicates(keep="first")
database_country=database_world["country"].drop_duplicates(keep="first")
database_sub_country=database_world[["subcountry","country"]].drop_duplicates(keep="first")
database_world.head()

,name,country,subcountry,geonameid
0,les Escaldes,Andorra,Escaldes-Engordany,3040051
1,Andorra la Vella,Andorra,Andorra la Vella,3041563
2,Umm Al Quwain City,United Arab Emirates,Imārat Umm al Qaywayn,290594
3,Ras Al Khaimah City,United Arab Emirates,Raʼs al Khaymah,291074
4,Zayed City,United Arab Emirates,Abu Dhabi,291580


2. Second Data Source

In [73]:
country_code=pd.read_csv("Country_Code.csv")
country_code=country_code[["name","alpha-3"]]
country_code.loc[country_code["name"]=="United States of America","name"]="United States"
country_code.head()

,name,alpha-3
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


3. Third Data Source

In [74]:
with open('US_States_Codes.json', 'r') as f:
    data = json.load(f)
us_state_codes = pd.DataFrame({'count': data}).reset_index()
us_state_codes.head()

,index,count
0,AK,Alaska
1,AL,Alabama
2,AR,Arkansas
3,AS,American Samoa
4,AZ,Arizona


In [75]:
dataframe_temp_2=Monthly_level_Data[["Date_Monthly_Level","Region","Users_1","Signups_1","Sessions_1","Pageviews_1","Bounces_1"]]
dataframe_temp_2=dataframe_temp_2.explode(["Region","Users_1","Signups_1","Sessions_1","Pageviews_1","Bounces_1"])
dataframe_temp_2=dataframe_temp_2.groupby(["Region"]).sum().reset_index()
dataframe_temp_2=dataframe_temp_2[dataframe_temp_2["Region"]!="(not set)"]
dataframe_temp_2.drop(columns=["Date_Monthly_Level"],inplace=True)
dataframe_temp_2=dataframe_temp_2.sort_values(by=["Users_1"],ascending=False)
dataframe_temp_2

,Region,Users_1,Signups_1,Sessions_1,Pageviews_1,Bounces_1
307,Texas,810960.0,49935.0,1122919.0,3495880.0,489322.0
97,Florida,448167.0,27372.0,618123.0,1938117.0,267603.0
50,California,431818.0,24359.0,589481.0,1791779.0,259638.0
104,Georgia,427377.0,24069.0,579592.0,1857872.0,256416.0
192,North Carolina,392998.0,23437.0,523256.0,1614914.0,228671.0
...,...,...,...,...,...,...
238,Riga,26.0,0.0,26.0,26.0,0.0
165,Matagalpa,26.0,0.0,26.0,53.0,0.0
124,Humacao,26.0,0.0,26.0,26.0,26.0
241,Riyadh Province,26.0,0.0,26.0,26.0,26.0


In [76]:
dataframe_temp_2=dataframe_temp_2.groupby(["Region"]).sum().reset_index()
dataframe_temp_2=dataframe_temp_2[dataframe_temp_2["Region"]!="(not set)"]
dataframe_temp_2=dataframe_temp_2.sort_values(by=["Region"],ascending=False)


In [77]:
#Dataframe Part 1
dataframe_part_1=pd.merge(dataframe_temp_2,database_sub_country,left_on="Region",right_on="subcountry")
dataframe_part_1=dataframe_part_1.drop_duplicates(subset=["Region"],keep=False)
dataframe_part_1=dataframe_part_1[["Region","country","Users_1","Signups_1","Sessions_1","Pageviews_1","Bounces_1"]]
dataframe_part_1.rename(columns={
    "Region":"Region",
    "country":"country"
},inplace=True)

In [78]:
#Dataframe Part 2
dataframe_part_2=pd.merge(dataframe_temp_2,database_name,left_on="Region",right_on="name")
dataframe_part_2=dataframe_part_2.drop_duplicates(subset=["Region"],keep=False)
dataframe_part_2=dataframe_part_2[["Region","country","Sessions_1","Users_1"]]

In [79]:
#Dataframe Part 1
dataframe_part_1=pd.merge(dataframe_temp_2,database_sub_country,left_on="Region",right_on="subcountry")
dataframe_part_1=dataframe_part_1.drop_duplicates(subset=["Region"],keep=False)
dataframe_part_1=dataframe_part_1[["Region","country","Users_1","Signups_1","Sessions_1","Pageviews_1","Bounces_1"]]
dataframe_part_1.rename(columns={
    "Region":"Region",
    "country":"country"
},inplace=True)
#Dataframe Part 2
dataframe_part_2=pd.merge(dataframe_temp_2,database_name,left_on="Region",right_on="name")
dataframe_part_2=dataframe_part_2.drop_duplicates(subset=["Region"],keep=False)
dataframe_part_2=dataframe_part_2[["Region","country","Users_1","Signups_1","Sessions_1","Pageviews_1","Bounces_1"]]

In [80]:
merge_1=pd.concat([dataframe_part_1,dataframe_part_2])
merge_1=pd.merge(merge_1,country_code,left_on=["country"],right_on=["name"],how="left")
merge_1=merge_1[["Region","country","alpha-3","Users_1","Signups_1","Sessions_1","Pageviews_1","Bounces_1"]]
merge_1_country_level=merge_1.groupby(["country"]).sum().reset_index()
merge_1_country_level.head()

,country,Users_1,Signups_1,Sessions_1,Pageviews_1,Bounces_1
0,Antigua and Barbuda,26.0,0.0,26.0,26.0,26.0
1,Australia,416.0,0.0,523.0,950.0,235.0
2,Austria,105.0,0.0,396.0,423.0,370.0
3,Bahamas,130.0,0.0,157.0,1295.0,52.0
4,Bahrain,26.0,0.0,26.0,53.0,0.0


In [81]:
merge_1_country_level.to_csv("Merged_Country_level.csv",index=False)

## World

In [120]:
# Create a world map using Plotly Express
fig = px.choropleth(
    data_frame=merge_1_country_level[merge_1_country_level["Users_1"]!=0],
    locations='country',
    locationmode='country names',
    color='Users_1',
    color_continuous_scale='Viridis',
    title='Users',
    range_color=(0,1000000)
)

# Show the plot
fig.show()

In [121]:
# Create a world map using Plotly Express
fig = px.choropleth(
    data_frame=merge_1_country_level[merge_1_country_level["Signups_1"]!=0],
    locations='country',
    locationmode='country names',
    color='Signups_1',
    color_continuous_scale='Viridis',
    title='Sign Ups',
    range_color=(0,450000)
)
# Show the plot
fig.show()

## United States

In [82]:
merge_2=merge_1.sort_values(by=["Users_1"],ascending=False)
merge_2=merge_2[merge_2["alpha-3"]=="USA"]
merge_2.drop(columns=["country","alpha-3"],inplace=True)
merge_2=pd.merge(merge_2,us_state_codes,left_on=["Region"],right_on=["count"])
merge_2=merge_2[["index","Users_1","Signups_1","Sessions_1","Pageviews_1","Bounces_1"]]

In [87]:
merge_2

,index,Users_1,Signups_1,Sessions_1,Pageviews_1,Bounces_1
0,TX,810960.0,49935.0,1122919.0,3495880.0,489322.0
1,CA,431818.0,24359.0,589481.0,1791779.0,259638.0
2,GA,427377.0,24069.0,579592.0,1857872.0,256416.0
3,NC,392998.0,23437.0,523256.0,1614914.0,228671.0
4,NY,389476.0,20710.0,549386.0,1835865.0,234359.0
5,IL,381040.0,20791.0,534891.0,1776621.0,228142.0
6,OH,346129.0,20683.0,480464.0,1538561.0,206561.0
7,PA,335549.0,18750.0,458670.0,1461833.0,196538.0
8,MI,277838.0,14651.0,400511.0,1323006.0,167417.0
9,VA,264218.0,12060.0,350467.0,1125117.0,165856.0


In [83]:
import plotly.express as px
fig = px.choropleth(merge_2,
                    locations='index', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='Sessions_1',
                    color_continuous_scale="Viridis_r", 
                    
                    )
fig.show()